# 01 — Data Loading & Scope
Notebook ini berisi proses pengolahan data emisi karbon.
Langkah yang dilakukan adalah:
* Mengambil data emisi karbon dari sumber data (format CSV)
* Melihat gambaran umum data (jumlah baris/kolom, contoh isi data, dan tipe data)
* Setelah data dipahami, saya melakukan filter data menjadi negara ASEAN dan rentang tahun 2000–2024

Tujuan dari bagian ini adalah memastikan dataset sudah berhasil dipanggil dengan benar sebelum masuk ke tahap analisis lebih lanjut.

In [19]:
import pandas as pd
import numpy as np

## Data Collection
Dataset emisi karbon diambil dari file CSV yang disimpan di GitHub.
Setelah data dipanggil menjadi dataframe, beberapa baris awal ditampilkan untuk memastikan data terbaca dengan benar.


In [20]:
df = pd.read_csv("https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv", delimiter=',')
df.head()


,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1750,AFG,2802560.0,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1751,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1752,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1753,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1754,AFG,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dataset ini berisi informasi emisi untuk banyak negara dengan rentang tahun 1750–2024.

Sebelum melakukan filter, dilakukan pengecekan jumlah baris dan kolom.

In [21]:
df.shape

(50411, 79)

Dataset memiliki 50.195 baris dan 79 kolom.
Jumlah ini mencakup seluruh negara dan rentang waktu panjang.

Untuk memahami struktur data, dicek daftar kolom yang tersedia.

In [22]:
df.columns


Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

Kolom yang tersedia mencakup informasi negara, tahun, serta berbagai metrik emisi karbon dan gas rumah kaca lainnya.

Selanjutnya dicek informasi tipe data dan missing value secara ringkas.

In [23]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50411 entries, 0 to 50410
Data columns (total 79 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    50411 non-null  object 
 1   year                                       50411 non-null  int64  
 2   iso_code                                   42480 non-null  object 
 3   population                                 41167 non-null  float64
 4   gdp                                        15251 non-null  float64
 5   cement_co2                                 29173 non-null  float64
 6   cement_co2_per_capita                      25648 non-null  float64
 7   co2                                        29384 non-null  float64
 8   co2_growth_abs                             27216 non-null  float64
 9   co2_growth_prct                            26239 non-null  float64
 10  co2_including_luc     

Terdapat kombinasi kolom numerik dan kategorikal.
Beberapa kolom memiliki missing value, yang akan dievaluasi lebih lanjut pada tahap data quality check

## Define Categorical Columns (Deterministik)
Kolom kategorikal diidentifikasi dan diurutkan secara deterministik agar hasil konsisten saat notebook dijalankan ulang.

In [24]:
cat_cols = sorted(df.select_dtypes(include="object").columns.tolist())
cat_cols


['country', 'iso_code']

### Filter Negara ASEAN
Scope analisis dibatasi pada negara anggota ASEAN.

In [25]:
asean_countries = [
    "Brunei",
    "Cambodia",
    "Indonesia",
    "Laos",
    "Malaysia",
    "Myanmar",
    "Philippines",
    "Singapore",
    "Thailand",
    "Vietnam"
]

df = df[df["country"].isin(asean_countries)].copy()


Dataset difilter agar hanya memuat negara ASEAN.

### Filter Tahun 2000–2024
Periode analisis dibatasi pada tahun 2000 hingga 2024.

In [26]:
df = df[(df["year"] >= 2000) & (df["year"] <= 2024)].copy()
df.shape


(250, 79)

In [27]:
df.head()


,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
7638,Brunei,2000,BRN,326429.0,NaN,0.0,0.0,5.886,-0.092,-1.537,...,NaN,0.017,0.0,0.0,0.0,0.0,9.218,8.123,-2.344,-39.830
7639,Brunei,2001,BRN,333353.0,NaN,0.0,0.0,5.758,-0.128,-2.178,...,NaN,0.017,0.0,0.0,0.0,0.0,9.554,8.428,-2.237,-38.852
7640,Brunei,2002,BRN,340108.0,NaN,0.0,0.0,5.285,-0.473,-8.206,...,NaN,0.017,0.0,0.0,0.0,0.0,8.517,7.424,-1.717,-32.479
7641,Brunei,2003,BRN,346650.0,NaN,0.0,0.0,6.140,0.854,16.162,...,NaN,0.018,0.0,0.0,0.0,0.0,9.600,8.381,-0.516,-8.412
7642,Brunei,2004,BRN,352921.0,NaN,0.0,0.0,5.967,-0.173,-2.817,...,NaN,0.018,0.0,0.0,0.0,0.0,8.826,7.879,-0.508,-8.519


Contoh baris data ditampilkan untuk memastikan hasil filter sesuai.

### Validasi Dasar dan Simpan Output Processed
Validasi dasar dilakukan untuk memastikan konsistensi data setelah filter.
Output processed disimpan agar notebook lanjutan menggunakan sumber data yang sama

In [28]:
from pathlib import Path

print("Countries:", sorted(df["country"].unique()))
print("Year min/max:", int(df["year"].min()), int(df["year"].max()))
print("Rows:", len(df), "Cols:", df.shape[1])

Countries: ['Brunei', 'Cambodia', 'Indonesia', 'Laos', 'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Vietnam']
Year min/max: 2000 2024
Rows: 250 Cols: 79


menampilkan daftar negara unik yang diurutkan, rentang tahun minimum–maksimum, serta jumlah baris dan kolom pada df.

In [29]:
dup_country_year = df.duplicated(["country", "year"]).sum()
print("Duplicate country-year:", int(dup_country_year))

Duplicate country-year: 0


menghitung jumlah baris dengan pasangan country–year yang duplikat lalu mencetak total duplikat tersebut sebagai bilangan bulat.

In [30]:
coverage = (
    df.groupby("country")["year"]
    .agg(["min", "max", "nunique"])
    .sort_values("nunique")
)
coverage

,min,max,nunique
country,,,
Brunei,2000,2024,25
Cambodia,2000,2024,25
Indonesia,2000,2024,25
Laos,2000,2024,25
Malaysia,2000,2024,25
Myanmar,2000,2024,25
Philippines,2000,2024,25
Singapore,2000,2024,25
Thailand,2000,2024,25


Membuat ringkasan per negara berisi tahun minimum, maksimum, dan jumlah tahun unik lalu mengurutkannya berdasarkan nunique (jumlah tahun unik) dari yang paling sedikit.

In [31]:
out_dir = Path("data/process")
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "owid_co2_asean_2000_2024.csv"
df.to_csv(out_path, index=False)

out_path.resolve()


PosixPath('/content/data/process/owid_co2_asean_2000_2024.csv')

Output menunjukkan dataset telah terfokus pada negara ASEAN dengan periode 2000–2024.
Dataset ini menjadi input utama untuk tahap data quality check dan EDA pada notebook berikutnya.

**Conclusion**  
Data emisi karbon berhasil dimuat dari OWID, lalu difilter pada negara ASEAN dan periode 2000–2024, divalidasi secara dasar (rentang tahun, duplikasi country–year, dan cakupan per negara), serta disimpan sebagai dataset terproses untuk digunakan pada tahap data quality check dan EDA berikutnya.